# Import Statements

In [1]:
import OCR_pipeline as OCR
import nltk
import spacy

# Download PunktSentenceTokenizer

In [24]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/kimberly/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

# Text Extraction

In [33]:
# textDictionary = OCR.extractTextFromAllPapers("Kenya")
# print(textDictionary)

textDictionary = {}
textDictionary['Agriculture.pdf'] = OCR.extractTextFromPaper("sample data/Kenya/AgricultureFisheriesandFoodAuthorityNo13of2013.PDF")
print(textDictionary)

Images converted to binary strings
Paper successfully converted!
{'Agriculture.pdf': ['SPECIAL ISSUE\n\nKenya Gazette Supplement No. 25 (Acts No. 13)\n\n \n\n \n\nREPUBLIC OF KENYA\n\nKENYA GAZETTE SUPPLEMENT\n\nACTS, 2013\n\nNAIROBI, 25th January, 2013\n\n \n\nCONTENT\n\nAct—\n\nThe Agriculture, Fisheries and Food Authority Act, 2013 .......ccccseseeees 183\n\nerence\n\nlawn\n\nNATIONAL COUNCIL FOR LAW REPORTING\n{\n| RECEIVED\n\nbodes\nbi\n\n \n  \n \n \n\n©, Box 10444-00700\nNAIROBI, KE\n\nYELL 2719231 FAK: 27126\neee nn\n\n=\n\nssamcns ane ten\n\nPRINTED AND PUBLISHED BY THE GOVERNMENT PRINTER, NAIROBI', 'THE AGRICULTURE, FISHERIES AND FOOD AUTHORITY\nACT\n\nNo. 13 of 2013\nDate of Assent: 14th January, 2013\nDate of Commencement: 25th January, 2013\nARRANGEMENT OF SECTIONS\n\nSection\nPART I—PRELIMINARY\n\n1—-Short title and commencement.\n\n2—-Interpretation.\n\nPART II—ESTABLISHMENT, FUNCTIONS AND POWERS OF\nTHE AUTHORITY\n\n3—Establishment of the Authority.\n4—Functions of the 

# Sentence Conversion Method

In [34]:
# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

In [45]:
# def remove_stopwords(texts):
#     return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

#returns list of cleaned sentences
def toSentences(pageList):
    
    # convert into long string (from list of page texts)
    longString = ''.join(pageList).replace('\n',' ')
    # split into list of sentences
    sentences = nltk.sent_tokenize(longString)
        
    # Remove Stop Words
    # sentences_nostops = remove_stopwords(sentences)
    
    # Form Bigrams
    sentences_bigrams = make_bigrams(sentences)

    # Initialize spacy 'en' model, keeping only tagger component (for efficiency)
    # python3 -m spacy download en
    nlp = spacy.load('en', disable=['parser', 'ner'])
    sentences_lemmatized = lemmatization(sentences_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
    
    return sent_tokenize(sentences_lemmatized)

# Cleaning

In [46]:
#convert each list of pages into one of cleaned sentences
# textDictionary = {}
for key in textDictionary:
    currentText = textDictionary[key]
    textDictionary[key] = toSentences(textDictionary[key])

print(textDictionary)

SPECIAL ISSUE  Kenya Gazette Supplement No. 25 (Acts No. 13)        REPUBLIC OF KENYA  KENYA GAZETTE SUPPLEMENT  ACTS, 2013  NAIROBI, 25th January, 2013     CONTENT  Act—  The Agriculture, Fisheries and Food Authority Act, 2013 .......ccccseseeees 183  erence  lawn  NATIONAL COUNCIL FOR LAW REPORTING { | RECEIVED  bodes bi            ©, Box 10444-00700 NAIROBI, KE  YELL 2719231 FAK: 27126 eee nn  =  ssamcns ane ten  PRINTED AND PUBLISHED BY THE GOVERNMENT PRINTER, NAIROBITHE AGRICULTURE, FISHERIES AND FOOD AUTHORITY ACT  No. 13 of 2013 Date of Assent: 14th January, 2013 Date of Commencement: 25th January, 2013 ARRANGEMENT OF SECTIONS  Section PART I—PRELIMINARY  1—-Short title and commencement.  2—-Interpretation.  PART II—ESTABLISHMENT, FUNCTIONS AND POWERS OF THE AUTHORITY  3—Establishment of the Authority. 4—Functions of the Authority. 5—Board of the Authority. 6— Powers of the Authority. 7—Conduct of business and affairs of the Authority. 8—Delegation by the Authority. 9—Remunerati

NameError: name 'bigram_mod' is not defined

# Topic Model Method

In [ ]:
#runs LDA on lemmatized text and returns corpus and LDA model
def LDA(lemmatized_sents):
    #create dictionary
    id2word = corpora.Dictionary(lemmatized_sents)
    
    # Creates corpus
    corpus = [id2word.doc2bow(lemmatized_sents) for lemmatized_sents in lemmatized_sents]
    
    lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=5, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)
    
    return [corpus, lda_model]
    


In [ ]:
# Print the Keywords in the 5 topics
#pprint(lda_model.print_topics())
#doc_lda = lda_model[corpus]